In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import os
print(os.listdir("../input"))
from scipy.stats import zscore
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import lightgbm as lgb
from bayes_opt import BayesianOptimization
import warnings, gc
import re

# Any results you write to the current directory are saved as output.

# Any results you write to the current directory are saved as output.

['2016-soi-tax-stats', 'individual-income-tax-statistics', 'midterm']
['2016-soi-tax-stats', 'individual-income-tax-statistics', 'midterm']
['2016-soi-tax-stats', 'individual-income-tax-statistics', 'midterm']


Using TensorFlow backend.


In [89]:
def multi_merge(left,right,*args):
    start = args[0]
    for i in range(1,len(args)):
        start = start.merge(args[i], how = 'left', left_on = left, right_on = right)
    return start

def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type not in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

def clean_data(df,year):
    column_list = list(df.columns)
    A_list = [column for column in column_list if re.match(r'^[A][0-9]{2,10}',column) or re.match(r'^[a][0-9]{2,10}',column)]
    N_list = [column for column in column_list if re.match(r'^[N][0-9]{2,10}',column) or re.match(r'^[n][0-9]{2,10}',column)]
    O_list = [column for column in column_list if (column not in A_list and column not in N_list and column not in ['STATE','STATEFIPS','state','statefips'])]
    if 'n1' in O_list:
        N1 = 'n1'
    else:
        N1 = 'N1'
    special = []
    pair = []
    for name in A_list:
        if 'N' + name[1:] not in N_list and 'n' + name[1:] not in N_list:
            special.append(name)
        else:
            pair.append(name)
            if 'n' + name[1:] in N_list:
                pair.append('n' + name[1:])
            else:
                pair.append('N' + name[1:])
    special = O_list + special
    pair = pair + ['zipcode',N1]
    df_special = df.loc[:,special]
    df_pair = df.loc[:,pair]
    for column in list(df_special.columns):
        if column != 'zipcode':
            df_special[column] = df_special[column]/df_special[N1]
    df_special = df_special.groupby(by='zipcode').agg('mean')
    #print(df_special)
    for name in N_list:
        if name[0] == 'N':
            A_name = 'A' + name[1:]
        else:
            A_name = 'a' + name[1:]
        df_pair[A_name] = df_pair[A_name]/df_pair[name]
        df_pair[name] = df_pair[name]/df_pair[N1]
    #print(df_pair)
    df_pair = df_pair.groupby(by='zipcode').mean().drop(N1,axis=1)
    #print(df_pair)
    df_result = df_special.merge(df_pair,how='inner',left_on='zipcode',right_on='zipcode').drop([N1,'agi_stub'],axis=1)
    #print(df_pair.shape)
    #print(df_special.shape)
    #print(df_result.shape)
    result_name_list = [name  if name == 'zipcode' else name + '_' + year for name in list(df_result.columns)]
    df_result.columns = result_name_list
    return df_result

In [85]:
df_2016 = pd.read_csv('../input/2016-soi-tax-stats/16zpallagi.csv')
df_2016 = df_2016.loc[(df_2016['zipcode'] != 0) & (df_2016['zipcode'] != 99999)].reset_index(drop=True)
df_2015 = pd.read_csv('../input/individual-income-tax-statistics/15zpallagi.csv')
df_2015 = df_2015.loc[(df_2015['zipcode'] != 0) & (df_2015['zipcode'] != 99999)].reset_index(drop=True)
df_2014 = pd.read_csv('../input/individual-income-tax-statistics/2014.csv')
df_2014 = df_2014.loc[(df_2014['zipcode'] != 0) & (df_2014['zipcode'] != 99999)].reset_index(drop=True)
df_2013 = pd.read_csv('../input/individual-income-tax-statistics/2013.csv')
df_2013 = df_2013.loc[(df_2013['zipcode'] != 0) & (df_2013['zipcode'] != 99999)].reset_index(drop=True)

In [82]:
df_2014.head()

,statefips,state,zipcode,agi_stub,n1,mars1,mars2,mars4,prep,n2,numdep,total_vita,vita,tce,a00100,n02650,a02650,n00200,a00200,n00300,a00300,n00600,a00600,n00650,a00650,n00700,a00700,n00900,a00900,n01000,a01000,n01400,a01400,n01700,a01700,schf,n02300,a02300,n02500,a02500,...,a07230,n07240,a07240,n07220,a07220,n07260,a07260,n09400,a09400,n85770,a85770,n85775,a85775,n09750,a09750,n10600,a10600,n59660,a59660,n59720,a59720,n11070,a11070,n10960,a10960,n11560,a11560,n06500,a06500,n10300,a10300,n85530,a85530,n85300,a85300,n11901,a11901,n11902,a11902,year
0,25,MA,1001,1,2890,2300,290,260,1470,3130,610,160,50,120,33609,2890,34289,2090,23034,690,407,370,486,330,309,90,36,330,2621,240,244,340,1773,570,4605,0,170,667,190,301,...,63,120,19,60,21,0,0,270,402,0,0,0,0,80,7,2520,4292,660,1311,550,1136,290,323,160,137,0,0,1120,729,1360,1161,0,0,0,0,270,220,2340,3334,2014
1,25,MA,1001,2,2210,1360,490,320,1210,3620,930,50,0,50,82123,2210,83204,1830,61608,660,385,340,675,320,422,330,140,220,2993,230,440,280,2453,550,9421,0,200,1272,460,2918,...,187,210,37,350,334,50,9,190,536,0,0,0,0,80,14,2180,9690,360,687,260,499,240,317,120,116,0,0,1910,5256,2010,5873,0,0,0,0,280,385,1900,4163,2014
2,25,MA,1001,3,1550,750,590,170,930,2940,790,0,0,0,95040,1550,96087,1360,73264,640,463,360,867,340,609,460,216,170,1867,240,814,200,2470,400,9779,0,120,734,340,4731,...,143,60,10,350,514,60,13,130,383,0,0,0,0,40,16,1540,11870,0,0,0,0,40,56,80,79,0,0,1500,8627,1510,9075,0,0,0,0,300,565,1230,3297,2014
3,25,MA,1001,4,920,240,600,70,560,2130,610,0,0,0,78961,920,79716,820,60764,440,240,270,828,260,557,390,219,130,1644,200,850,150,2474,290,7418,0,90,632,220,3847,...,154,0,0,240,397,40,9,110,306,0,0,0,0,0,0,910,9955,0,0,0,0,0,0,80,87,0,0,910,7858,910,8240,0,0,0,0,200,501,710,2202,2014
4,25,MA,1001,5,1160,190,920,40,770,3000,930,0,0,0,151278,1160,152951,1080,119808,700,770,440,1740,420,1328,680,433,190,4550,320,2532,160,3113,370,12253,0,80,603,200,4182,...,216,0,0,230,279,70,39,140,623,0,0,0,0,0,0,1150,22283,0,0,0,0,0,0,130,130,0,0,1150,20025,1150,20807,0,0,0,0,380,1548,750,2929,2014


In [34]:
df_2016_result = clean_data(df_2016,'2016').fillna(0)
df_2016_result.head()

,mars1_2016,MARS2_2016,MARS4_2016,PREP_2016,N2_2016,NUMDEP_2016,TOTAL_VITA_2016,VITA_2016,TCE_2016,VITA_EIC_2016,RAL_2016,RAC_2016,ELDERLY_2016,SCHF_2016,A00100_2016,A00101_2016,A02650_2016,N02650_2016,A00200_2016,N00200_2016,A00300_2016,N00300_2016,A00600_2016,N00600_2016,A00650_2016,N00650_2016,A00700_2016,N00700_2016,A00900_2016,N00900_2016,A01000_2016,N01000_2016,A01400_2016,N01400_2016,A01700_2016,N01700_2016,A02300_2016,N02300_2016,A02500_2016,N02500_2016,...,A07230_2016,N07230_2016,A07240_2016,N07240_2016,A07220_2016,N07220_2016,A07260_2016,N07260_2016,A09400_2016,N09400_2016,A85770_2016,N85770_2016,A85775_2016,N85775_2016,A09750_2016,N09750_2016,A10600_2016,N10600_2016,A59660_2016,N59660_2016,A59720_2016,N59720_2016,A11070_2016,N11070_2016,A10960_2016,N10960_2016,A11560_2016,N11560_2016,A06500_2016,N06500_2016,A10300_2016,N10300_2016,A85530_2016,N85530_2016,A85300_2016,N85300_2016,A11901_2016,N11901_2016,A11902_2016,N11902_2016
zipcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,838.333333,496.666667,141.666667,830.000000,2526.666667,651.666667,31.666667,6.666667,26.666667,0.000000,6.666667,141.666667,440.000000,0.0,84386.333333,48396.833333,105.565148,1.0,83.229204,0.863023,1.044836,0.438299,3.228680,0.285814,2.602526,0.262766,0.660451,0.339496,15.868968,0.127038,8.212946,0.225192,15.118106,0.142870,30.963139,0.236359,6.608240,0.049060,13.560271,0.184863,...,1.202195,0.063188,0.170581,0.032847,1.095317,0.139895,0.582708,0.028811,3.496698,0.115189,1.856667,0.018044,2.039881,0.026955,0.669630,0.013885,17.281488,0.974309,1.896032,0.063525,1.915203,0.050867,1.370676,0.037105,0.946273,0.052613,0.571667,0.007296,15.989066,0.871976,16.653966,0.891167,0.871429,0.072917,1.271429,0.072917,4.133131,0.246470,3.269088,0.717141
1002,848.333333,588.333333,125.000000,770.000000,2703.333333,688.333333,53.333333,35.000000,18.333333,13.333333,6.666667,116.666667,520.000000,5.0,133195.833333,102969.500000,124.362069,1.0,83.463728,0.791230,1.580278,0.559794,9.671493,0.423521,8.080448,0.402024,0.957778,0.268201,22.893106,0.267004,10.794825,0.380736,16.282392,0.196638,38.769604,0.275611,6.423889,0.029374,13.053997,0.235832,...,1.074280,0.095432,0.208772,0.036857,1.031757,0.110728,2.352333,0.044139,3.922250,0.220526,2.132065,0.029595,2.203904,0.035730,0.636667,0.013871,23.304370,0.965453,1.819526,0.062214,1.874325,0.050879,1.305534,0.037454,0.993595,0.057743,0.775325,0.012244,19.892974,0.854636,21.251908,0.882041,1.365714,0.072016,2.075000,0.090535,4.882348,0.279957,5.640635,0.636651
1003,85.000000,0.000000,0.000000,10.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1217.500000,0.000000,36.695000,1.0,31.738333,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.132143,0.966667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.800000,0.100000,0.000000,0.000000,3.270000,0.666667,3.365000,0.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.793750,0.533333
1005,195.000000,158.333333,35.000000,211.666667,733.333333,206.666667,0.000000,0.000000,0.000000,0.000000,0.000000,46.666667,106.666667,0.0,22972.000000,13083.000000,104.713382,1.0,77.924632,0.906335,0.613588,0.455441,2.236117,0.162791,1.622363,0.162791,0.533838,0.320528,17.250556,0.123536,4.763333,0.124410,11.832667,0.096273,23.702714,0.183537,6.303667,0.073963,10.703957,0.140725,...,1.215833,0.070737,0.143750,0.030754,1.461111,0.158398,1.200000,0.034685,3.220524,0.099778,1.900000,0.018849,1.583333,0.024802,0.533333,0.017857,16.012454,0.977315,1.776389,0.063823,1.903571,0.050265,1.227381,0.034722,0.914583,0.050558,0.833333,0.006944,14.706191,0.873611,15.696917,0.895304,0.000000,0.000000,0.000000,0.000000,4.

In [90]:
df_2014_result = clean_data(df_2014,'2014').fillna(0)
df_2014_result.head()

,mars1_2014,mars2_2014,mars4_2014,prep_2014,n2_2014,numdep_2014,total_vita_2014,vita_2014,tce_2014,schf_2014,year_2014,a00100_2014,a00101_2014,a02650_2014,n02650_2014,a00200_2014,n00200_2014,a00300_2014,n00300_2014,a00600_2014,n00600_2014,a00650_2014,n00650_2014,a00700_2014,n00700_2014,a00900_2014,n00900_2014,a01000_2014,n01000_2014,a01400_2014,n01400_2014,a01700_2014,n01700_2014,a02300_2014,n02300_2014,a02500_2014,n02500_2014,a26270_2014,n26270_2014,a02900_2014,...,a07230_2014,n07230_2014,a07240_2014,n07240_2014,a07220_2014,n07220_2014,a07260_2014,n07260_2014,a09400_2014,n09400_2014,a85770_2014,n85770_2014,a85775_2014,n85775_2014,a09750_2014,n09750_2014,a10600_2014,n10600_2014,a59660_2014,n59660_2014,a59720_2014,n59720_2014,a11070_2014,n11070_2014,a10960_2014,n10960_2014,a11560_2014,n11560_2014,a06500_2014,n06500_2014,a10300_2014,n10300_2014,a85530_2014,n85530_2014,a85300_2014,n85300_2014,a11901_2014,n11901_2014,a11902_2014,n11902_2014
zipcode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,811.666667,500.000000,143.333333,840.000000,2535.000000,666.666667,35.000000,8.333333,28.333333,0.000000,2014.0,81489.333333,47129.333333,109.140110,1.0,82.011926,0.864055,0.994513,0.472002,3.864726,0.308931,2.941750,0.287169,0.567733,0.325671,13.824569,0.104752,7.432729,0.228090,16.129167,0.134614,26.254344,0.267502,6.191984,0.065589,13.939793,0.173020,31.623029,0.078226,5.407742,...,1.167388,0.072168,0.167063,0.029209,1.128013,0.144014,0.294702,0.027526,3.981319,0.106143,0.000000,0.000000,0.0,0.0,0.220833,0.014948,18.286445,0.972076,1.947348,0.065212,1.992343,0.051326,1.278209,0.039125,0.979583,0.060050,0.0,0.00000,16.795933,0.866675,17.573359,0.889132,1.140000,0.055556,1.450000,0.066667,4.541972,0.237553,3.656067,0.713543
1002,873.333333,585.000000,126.666667,785.000000,2756.666667,715.000000,60.000000,41.666667,18.333333,6.666667,2014.0,130828.500000,99734.166667,126.706004,1.0,81.653979,0.793951,1.687661,0.572400,10.265124,0.436344,8.140574,0.413597,0.934066,0.266705,28.122463,0.265028,23.085121,0.395551,15.263862,0.182479,37.424042,0.275674,6.009205,0.032370,12.608901,0.220233,29.537108,0.103670,6.706211,...,1.052021,0.101393,0.196078,0.032693,1.046855,0.112356,1.800857,0.055640,4.163693,0.220056,2.166667,0.001425,0.0,0.0,0.349167,0.011564,22.653290,0.966391,1.856548,0.062613,1.913584,0.051708,1.317206,0.042894,0.970733,0.064791,1.8,0.00095,20.099154,0.851125,21.680990,0.879104,1.265517,0.064444,4.430435,0.102222,5.031736,0.295140,4.394931,0.627477
1005,183.333333,161.666667,31.666667,208.333333,703.333333,196.666667,0.000000,0.000000,0.000000,3.333333,2014.0,21970.500000,13145.500000,98.480548,1.0,77.865429,0.872110,0.885880,0.476990,5.635005,0.234662,2.551488,0.227162,0.598627,0.296038,13.286667,0.133598,9.478016,0.238562,15.642667,0.091269,19.883636,0.200094,6.177333,0.085632,11.662333,0.129535,24.520000,0.035539,3.099444,...,1.145333,0.074479,0.165556,0.031944,1.432629,0.159771,0.336667,0.035539,2.627571,0.107807,0.000000,0.000000,0.0,0.0,0.166667,0.014630,13.663960,0.970931,1.937054,0.060370,1.860714,0.052407,1.291667,0.033889,0.903333,0.062288,0.0,0.00000,13.105007,0.867228,13.556107,0.896852,0.000000,0.000000,0.000000,0.000000,4.307463,0.275114,2.797529,0.635559
1007,570.000000,546.666667,95.000000,630.000000,2333.333333,690.000000,23.333333,6.666667,16.666667,8.333333,2014.0,84382.666667,60142.333333,109.601405,1.0,84.216680,0.882596,0.941712,0.535412,3.766784,0.297225,2.845747,0.276254,0.739172,0.345257,15.035429,0.170834,11.878344,0.233553,17.785972,0.108426,29.273157,0.236949,6.536667,0.060096,12.477395,0.141042,39.467024,0.086975,4.932173,...,1.189242,0.082243,0.185556,0.033116,1.138948,0.163553,0.733788,0.049865,3.627411,0.149706,0.000000,0.000000,0.0,0.0,0.216667,0.010866,17.837765,0.976773,1.830237,0.059524,1.909427,0.048499,1.222407,0.037311,0.948078,0.068248,0.0,0.00000,16.598188,0.868500,17.523169,0.887649,1.063636,0.065476,1.730769,0.077381,5.127150,0.241219,3.583648,0.718598
1

In [7]:
def normalization(x):
    min_x = min(x)
    max_x = max(x)
    return np.array([(i-min_x)/(max_x-min_x) for i in x])

In [36]:
def zscore_transformation(df):
    for name in df.columns:
        df[name] = zscore(df[name])
    return df.dropna(axis=1).dropna(axis=0)

In [91]:
df_2016_result = zscore_transformation(df_2016_result)
df_2014_result = zscore_transformation(df_2014_result)

/opt/conda/lib/python3.6/site-packages/scipy/stats/stats.py:2253: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [92]:
df_2014_result.shape

(27679, 122)

In [93]:
df_train = pd.read_csv('../input/midterm/train.csv')
df_test = pd.read_csv('../input/midterm/test.csv')
df_train = multi_merge('zipcode','zipcode',df_train,df_2014_result).drop(['id','zipcode'],axis=1)
df_test = multi_merge('zipcode','zipcode',df_test,df_2014_result).drop(['id','zipcode'],axis=1)
x,y = to_xy(df_train,'score')

In [94]:
df_train.dropna(axis=1)
df_train.shape

(23788, 123)

In [98]:
kf = KFold(5)
oos_y = []
oos_pred = []
fold = 0
for train,test in kf.split(x):
    fold += 1
    print('Fold #{}'.format(fold))
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(20, input_dim=x.shape[1], activation='relu'))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-2, patience=20, verbose=1, mode='auto')
    checkpointer = ModelCheckpoint(filepath="midterm_model1_best", verbose=0, save_best_only=True)
    model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=0,epochs=250)
    
    pred = model.predict(x_test)
    oos_y.append(y_test)
    oos_pred.append(pred)
    score = np.sqrt(metrics.mean_squared_error(pred,y_test))
    print("Fold score (RMSE): {}".format(score))

oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
score = metrics.mean_squared_error(oos_pred,oos_y)
print("Final score (RMSE): {}".format(np.sqrt(score)))

df_test = pd.read_csv('../input/midterm/test.csv')
df_test = multi_merge('zipcode','zipcode',df_test,df_full_detail).drop(['id','zipcode'],axis=1)
true_test = df_test.values.astype(np.float32)
pred_test = model.predict(true_test)
pred_test = model.predict(true_test)
final_test_score = np.concatenate(pred_test)

df_test['score'] = final_test_score
df_test = df_test.loc[:,['id','score']]
df_test.to_csv('csv_to_submit_regularization.csv', index = False)

Fold #1
Epoch 00020: early stopping


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').